# **GPU with Keras**


## Author

<a href="https://www.linkedin.com/in/ganesh-kommana/" target="_blank">Ganesh Kommana</a>

You may have heard of GPUs (Graphics Processing Unit) and CPUs (Central Processing Unit), but what is the difference? GPUs have been commonly seen used by gamers for better visual rendering, but nowadays its applications extend way beyond improving videogame experience. With respect to deep learning, GPUs are extremely helpful by speeding up certain computations. The difference is evident especially for models that train on large datasets, in which the researcher can take advantage of parallel computing to run operations simultaneously and save time. In this lab, you will learn about how to utilize GPU for `tensorflow`, specifically `keras`.


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ML311-Coursera/labs/Module6/L1/img_GPU.jpeg" width="600" alt="computer components">
<center>


**_Note_**: Skills Network Labs currently doesn't have any GPUs available. In order to test the difference between CPU and GPU, please run this lab on a local machine or environment that has GPUs.


## Objectives

After completing this lab you will be able to:

 - Set environment to CPU/GPU
 - Control usage of CPU/GPU in parts of the code


----


### Installing Required Libraries



In [2]:
!pip install pandas
!pip install numpy
!pip install seaborn
!pip install matplotlib
!pip install scikit-learn


[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
# %%capture
!pip install --upgrade tensorflow -qqq

### Importing Required Libraries

_We recommend you import all required libraries in one place, as follows:_


In [4]:
import warnings
warnings.simplefilter('ignore')

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 

import numpy as np

import tensorflow as tf
# Import the keras library
from tensorflow import keras
from tensorflow.keras import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.python.client import device_lib

## Benefits of Using GPU


GPU excels in parallel computing in comparison to CPU. This technique is especially useful for deep learning algorithms, such as building a Convolutional Neural Network (CNN), or as a matter of fact, any neural network. An example of a parallel processing task is performing convolution on an input layer, in which the kernel is multiplied with the input layer matrix, one local region at a time.

The runtime difference is especially noticeable when you train a CNN with multiple epochs - tasks where a lot of matrix operations are involved!


## Using CPU


By default, `tensorflow` searches for available GPU to use. There are two ways to force your machine to ignore all GPUs and run code with CPU instead.


If you want the entire code/notebook to run on CPU, you can specify the environment _**before**_ importing tensorflow/keras. If you decide to switch back, you can restart the kernel and run import as usual without the line below.


In [5]:
# Specify the environment variable value
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

If the environment variable `CUDA_VISIBLE_DEVICES` value takes the values 0/1 (or other positive values), the machine is using GPU to run the code. By setting it to -1, it specifies the algorithm to be run with CPU.


In [6]:
# Check that CPU is used
print(os.environ['CUDA_VISIBLE_DEVICES'])

-1


If instead you want to use CPU for portions of the code in a notebook, consider the following approach. Here, you specify what to run with `/CPU:0` using a `with` statement. Using `%%timeit` with `-n1 -r1` will time the process for one pass of the cell. As an example, we'll be training the following CNN on a **DATASET**. Feel free to change the code within the statement to test CPU performance! 


In [7]:
# Import data
(X_train, y_train), (X_test, y_test) = keras.datasets.mnist.load_data()

In [8]:
# Reshape the data
X_train = X_train.reshape((X_train.shape[0],X_train.shape[1],X_train.shape[2],1))
X_test = X_test.reshape((X_test.shape[0],X_test.shape[1],X_test.shape[2],1))

y_train = y_train.reshape((y_train.shape[0],1))
y_test = y_test.reshape((y_test.shape[0],1))

In [9]:
%%timeit -n1 -r1
# Building the CNN model and fitting on train data
with tf.device('/CPU:0'):
    model_cpu = Sequential()
    model_cpu.add(Conv2D(input_shape = (28, 28, 1),
                     filters=5, 
                     padding='Same',
                     kernel_size=(3,3)
                     ))
    model_cpu.add(MaxPooling2D(pool_size=(2,2)))
    model_cpu.add(Flatten())
    model_cpu.add(Dense(256, activation='relu'))
    model_cpu.add(Dense(10, activation='softmax'))
    
    model_cpu.compile(optimizer='adam', 
              loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
    
    model_cpu.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=5)

Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 18s 6ms/step - accuracy: 0.9133 - loss: 0.8252 - val_accuracy: 0.9426 - val_loss: 0.2081
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 13s 7ms/step - accuracy: 0.9620 - loss: 0.1360 - val_accuracy: 0.9576 - val_loss: 0.1617
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 12s 7ms/step - accuracy: 0.9706 - loss: 0.1051 - val_accuracy: 0.9639 - val_loss: 0.1333
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 14s 7ms/step - accuracy: 0.9758 - loss: 0.0872 - val_accuracy: 0.9702 - val_loss: 0.1285
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 13s 7ms/step - accuracy: 0.9788 - loss: 0.0753 - val_accuracy: 0.9667 - val_loss: 0.1415
1min 10s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


## Using GPU


As mentioned above, `tensorflow` automatically searches for GPUs to run on. Let's take a closer look at how we can have more control over that.


### Check Availability


First, you can check the number of GPUs available on the machine. 


In [10]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


If you're running this notebook in Skills Network Lab, you can see that it doesn't have any GPUs available for use. However, if your local machine does have GPU(s), you can try the following code to play with what you want to run on GPU.


### Choosing Specific GPUs


In order to specify a particular GPU to run on, we have to first check what units there are in the environment. The following lists out the information of each device, including the device name, type, memory limit, and so on. 


In [11]:
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 6573538541962570668
xla_global_id: -1
]


To specify using a specific GPU, again use `tf.device()` with the `name` as input, just like we did for the CPU case. In the `with` statement, proceed with writing code as usual. Here, we are specifying `tensorflow` to be run on GPU ennumerated #2. We also use `%%timeit` here so you can compare the time that GPU took to run in comparison with CPU!


In [12]:
%%timeit -n1 -r1
with tf.device('/device:GPU:2'):
    model_gpu = Sequential()
    model_gpu.add(Conv2D(input_shape = (28, 28, 1),
                     filters=5, 
                     padding='Same',
                     kernel_size=(3,3)
                     ))
    model_gpu.add(MaxPooling2D(pool_size=(2,2)))
    model_gpu.add(Flatten())
    model_gpu.add(Dense(256, activation='relu'))
    model_gpu.add(Dense(10, activation='softmax'))
    
    model_gpu.compile(optimizer='adam', 
              loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
    
    model_gpu.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=5)

Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 14s 7ms/step - accuracy: 0.9168 - loss: 1.0695 - val_accuracy: 0.9429 - val_loss: 0.2334
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 12s 6ms/step - accuracy: 0.9630 - loss: 0.1397 - val_accuracy: 0.9499 - val_loss: 0.1839
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 13s 7ms/step - accuracy: 0.9704 - loss: 0.1041 - val_accuracy: 0.9654 - val_loss: 0.1346
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 13s 7ms/step - accuracy: 0.9743 - loss: 0.0908 - val_accuracy: 0.9656 - val_loss: 0.1430
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 13s 7ms/step - accuracy: 0.9788 - loss: 0.0735 - val_accuracy: 0.9636 - val_loss: 0.1562
1min 4s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


## Using CPU and GPU jointly


What if we want to use _both_ CPU and GPU for different parts of the same python script? Turns out we can do that too! Simply take advantage of the `tf.device()` function again to specify which unit the code fragment should be run on. Below, we show an example of how to run the same matrix operation on multiple GPUs and add up the tensors on CPU.


In [14]:
# Enable tensor allocations or operations to be printed
tf.debugging.set_log_device_placement(True)

# Get list of all logical GPUs
gpus = tf.config.list_logical_devices('GPU')

# Check if there are GPUs on this computer
if gpus:
  # Run matrix computation on multiple GPUs
    c = []
    for gpu in gpus:
        with tf.device(gpu.name):
            a = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
            b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]]) 
            c.append(tf.matmul(a, b))

    # Run on CPU 
    with tf.device('/CPU:0'):
        matmul_sum = tf.add_n(c)

    print(matmul_sum)